Esta notebook carga el .JSON creado por pptx_cierre_a_json (input) y genera un excel con la data ordenada y legible (output).
Hay distintas funciones que van extrayendo la data y limpiando los str.

In [3]:
import pandas as pd
import json
import re
# Use openpyxl to hide a column
from openpyxl import load_workbook

In [4]:
"""MUY IMPORTANTE EL PASO DE FILTERED_DATA, YA QUE LAS PRIMERAS 3 Y LA ÚLTIMA DIAPOSITIVA SON DE ESTILO, ES DECIR, NO HAY INFORMACIÓN SOBRE LOS PROYECTOS"""

# Load JSON data from a file
with open('../Data/2025-I+D_Ficha Registro Iniciativas.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Ignore the first 2 and last 2 elements
filtered_data = data[4:] #CHECKEAR QUE NO SE HAYA CAMBIADO EL FORMATO DE LA PPTX
# Create a DataFrame
df = pd.DataFrame(filtered_data)
#print(df)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/2025-I+D_Ficha Registro Iniciativas.json'

In [ ]:
"""Inicializo el df con los nombres de las columnas que quiero"""
df["Proyecto"]=None
df["Codigo"]=None
df["Equipo"]=None
df["Referente"]=None
df["Fecha_Inicio"]=None
df["Fecha_Cierre"]=None
df["Duracion_Semanas"]=None
df["Industria"]=None
df["Tipo_Cliente"]=None
df["Detalle_Cliente"]=None
df["Interlocutor"]=None
df["Celula"]=None
df["Docu_Validada"]=None
df["Objetivo"]=None
df["Alcance"]=None
df["Link_Presentacion"]=None
df["Link_Demo"]=None
df["Link_Codigo"]=None
df["Link_Documentacion"]=None
df["Tecnologias"]=None
#df["Comentarios"]=None


In [ ]:
# Function to remove specified strings and unwanted substrings (including '\u200b', '(Si/ No)', etc.)
def clean_elements(text_list):
    return [
        item.replace('\u200b', '')  # Remove zero-width space
            .replace('(Si/ No) :', '')
            .replace('(Si/ No):', '')  # Remove '(Si/ No)'
            .replace('(Si/ No)', '')
            .replace('(Interno/Externo/ I+D) :', '')
            .replace('(Interno/Externo/ I+D):', '')
            .replace('(Interno/Externo/ I+D)', '')
        for item in text_list
        if item != "Equipo I+D" 
        and item != 'FICHA de REGISTRO DE Iniciativas' 
        and not item.startswith("Copyright")
    ]

# Apply the function to the 'texts' column
df['texts'] = df['texts'].apply(clean_elements)

In [ ]:
# Function to extract the project name
def extract_proyecto(text_list):
    for item in text_list:
        if item.startswith("Nombre Iniciativa:"):
            return item.replace("Nombre Iniciativa:", "").strip()  # Remove "PROYECTO:" and strip whitespace
    return None  # Return None if not found

# Apply the function to the 'text' column to create the 'Proyecto' column
df['Proyecto'] = df['texts'].apply(extract_proyecto)

In [ ]:
# Function to extract and clean the team name
def extract_equipo(text_list):
    for item in text_list:
        if item.startswith("Equipo"):
            # Remove "Equipo:", "Nombre", and any colons, then strip whitespace
            cleaned_str = item.replace("Equipo :", "").strip()
            return cleaned_str
    return None  # Return None if not found

# Apply the function to the 'texts' column to create the 'Equipo' column
df['Equipo'] = df['texts'].apply(extract_equipo)

In [ ]:
def extract_codigo(text_list):
    for item in text_list:
        if item.startswith("CÓDIGO") or item.startswith("código"):
            cleaned_str = item.replace("código :", "").replace("código:", "").replace("CÓDIGO :", "").replace("CÓDIGO:", "").strip()
            return cleaned_str
    return None  # Return None if not found

# Apply the function to the 'texts' column to create the 'Equipo' column
df['Codigo'] = df['texts'].apply(extract_codigo)

In [ ]:
def extract_referente(text_list):
    for item in text_list:
        if item.startswith("Referente"):
            # Remove "Equipo:", "Nombre", and any colons, then strip whitespace
            cleaned_str = item.replace("Referente:", "").strip()
            return cleaned_str
    return None  # Return None if not found

# Apply the function to the 'texts' column to create the 'Equipo' column
df['Referente'] = df['texts'].apply(extract_referente)

In [ ]:
def extract_info_to_columns(df,text_column):
    keywords = [
        ('Industria:', 'Industria'),
        ('Tipo de Cliente:', 'Tipo_Cliente'),
        ('Detalle de cliente:', 'Detalle_Cliente'),
        ('Interlocutor de cliente:', 'Interlocutor'),
        ('Célula:','Celula'),
        ('Documentación Validada:','Docu_Validada')
    ]

    # Function to extract the first link for a given keyword from a list of strings
    def extract_info(text_list, keyword):
        for text in text_list:
            if text.startswith(keyword):
                return text.replace(keyword, "").strip()
        return None

    # Overwrite existing columns with extracted links
    for keyword, column in keywords:
        df[column] = df[text_column].apply(lambda x: extract_info(x, keyword))

    return df

df = extract_info_to_columns(df, 'texts')

In [ ]:
def alcance(text_list):
    for text in text_list:
        if text.startswith("Alcance"):
            return text.replace("Alcance:", "")
    return None

# Create the "Tecnologias" column
df['Alcance'] = df['texts'].apply(alcance)

In [ ]:
def objetivos(text_list):
    for text in text_list:
        if text.startswith("Objetivo"):
            return text.replace("Objetivo:", "")
    return None

# Create the "Tecnologias" column
df['Objetivo'] = df['texts'].apply(objetivos)

In [ ]:
# Function to extract relevant technology information
def extract_tecnologias(text_list):
    for text in text_list:
        if text.startswith("TECNOLOGIAS APLICADAS"):
            return text.replace("TECNOLOGIAS APLICADAS", "")
    return None

# Create the "Tecnologias" column
df['Tecnologias'] = df['texts'].apply(extract_tecnologias)

In [ ]:
def extract_dates_and_quantity(df,text_column):
    # Define keywords and corresponding column names
    keywords = [
        ('Fecha Inicio:', 'Fecha_Inicio'),
        ('Fecha Fin:', 'Fecha_Cierre'),
        ('Cantidad semanas netas :', 'Duracion_Semanas'),
    ]

    # Function to extract the first link for a given keyword from a list of strings
    def extract_dates(text_list, keyword):
        for text in text_list:
            if text.startswith(keyword):
                return text.replace(keyword, "").strip()
        return None

    # Overwrite existing columns with extracted links
    for keyword, column in keywords:
        df[column] = df[text_column].apply(lambda x: extract_dates(x, keyword))

    return df

df = extract_dates_and_quantity(df, 'texts')


In [ ]:
def extract_links_to_columns(df, text_column):
    # Define keywords and corresponding column names
    keywords = [
        ('LINK ACCESO A PRESENTACION', 'Link_Presentacion'),
        ('LINK ACCESO A CODIGO', 'Link_Codigo'),
        ('LINK ACCESO A DEMO grabada (si corresponde)', 'Link_Demo'),
        ('LINK ACCESO A DOCUMENTACION', 'Link_Documentacion')
    ]

    # Function to extract the first link for a given keyword from a list of strings
    def extract_link(text_list, keyword):
        for text in text_list:
            if keyword.lower() in text.lower():  # Case insensitive match for the keyword
                # Look for a URL in the text
                match = re.search(r'https?://[^\s]+', text)
                if match:
                    return match.group(0)
                else:
                    # Look for the "no aplica" expression in different cases (case insensitive)
                    no_aplica = re.search(r'no aplica', text, re.IGNORECASE)
                    if no_aplica:
                        return no_aplica.group(0)  # Return the exact match for "no aplica"
        return None  # Return None if no match is found

    # Overwrite existing columns with extracted links
    for keyword, column in keywords:
        df[column] = df[text_column].apply(lambda x: extract_link(x, keyword))

    return df

df = extract_links_to_columns(df, 'texts')

In [ ]:
#def comentarios(text_list):
    #for text in text_list:
        #if text.startswith("Comentarios adicionales (opcional)"):
            #return text.replace("Comentarios adicionales (opcional) :", "")
    #return None

# Create the "Tecnologias" column
#df['Comentarios'] = df['texts'].apply(comentarios)

In [ ]:
df=df.drop("texts",axis=1)

In [ ]:
# Save to Excel
excel_file = '../Data/output_proyectos.xlsx'
df.to_excel(excel_file, index=False)

# Load the workbook and select the active sheet
wb = load_workbook(excel_file)
ws = wb.active

# Save the changes
wb.save(excel_file)
wb.close()

print(f"DataFrame saved to {excel_file}")

DataFrame saved to ../Data/output_proyectos.xlsx
